In [ ]:
# 对大模型SQL结果进行评测
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import traceback

def get_db_engine(database, host="rm-uf6z891lon6dxuqblqo.mysql.rds.aliyuncs.com", user="student123", password="student321", port=3306):
    """创建数据库连接引擎"""
    # 需要安装依赖包pip install mysql-connector-python -i https://pypi.tuna.tsinghua.edu.cn/simple
    connection_str = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'
    engine = create_engine(connection_str)
    return engine

def get_db_session(engine):
    """创建数据库会话"""
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

def get_markdown_result(session, sql):
    """
    执行SQL并返回markdown格式的结果
    返回值：
    - isok: 'Yes'表示执行成功，'No'表示执行失败
    - result: 成功时返回markdown表格，失败时返回错误信息
    """
    try:
        # 执行SQL查询
        result = session.execute(text(sql))
        
        # 获取列名
        columns = result.keys()
        
        # 获取所有数据
        rows = result.fetchall()
        
        if not rows:
            return 'Yes', '查询结果为空'
            
        # 构建markdown表格
        markdown = '| ' + ' | '.join(columns) + ' |\n'
        markdown += '| ' + ' | '.join(['---' for _ in columns]) + ' |\n'
        
        # 添加数据行
        for row in rows:
            markdown += '| ' + ' | '.join(str(cell) for cell in row) + ' |\n'
            
        return 'Yes', markdown
        
    except Exception as e:
        error_msg = str(e)
        traceback_msg = traceback.format_exc()
        return 'No', f'SQL执行错误: {error_msg}'
    finally:
        session.close()

filename = './QA/sql_result_qwen_turbo.xlsx'

# 获取数据库句柄
engine = get_db_engine(database="life_insurance")

# 读取待评测文件
df = pd.read_excel(filename)

df['能否运行'] = 0
markdown_list = []
for index, row in df.iterrows():
    sql = row['SQL']
    print(sql)
    session = get_db_session(engine)
    if str(sql) == 'nan':
        df.loc[index, '能否运行'] = 'No 没有找到SQL'
        continue
        
    # 如果有多个sql，只执行第一个
    sqls = sql.split(';')
    sql = sqls[0]        
    isok, markdown_table = get_markdown_result(session, sql)
    markdown_list.append(markdown_table)
    if isok == 'Yes':
        df.loc[index, '能否运行'] = 'Yes'
    else:
        df.loc[index, '能否运行'] = 'No ' + markdown_table
        
df.to_excel(filename, index=False)
df